In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd
from bs4 import BeautifulSoup
import requests
from pathlib import Path
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.



WARNING conda.base.context:use_only_tar_bz2(632): Conda is constrained to only using the old .tar.bz2 file format because you have conda-build installed, and it is <3.18.3.  Update or remove conda-build to get smaller downloads and faster extractions.


Solving environment: ...working... done

# All requested packages already installed.



WARNING conda.base.context:use_only_tar_bz2(632): Conda is constrained to only using the old .tar.bz2 file format because you have conda-build installed, and it is <3.18.3.  Update or remove conda-build to get smaller downloads and faster extractions.


Libraries imported.


## Scrap data from Wikipedia page into a DataFrame

In [42]:
postal_codes_can=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [43]:
soup=BeautifulSoup(postal_codes_can,'lxml')

In [44]:
postcode=[]
borough=[]
neighbourhood=[]
i=1
for tag in soup.table.find_all('td'):
    if i == 1:
        postcode.append(tag.text)
    if i == 2:
        borough.append(tag.text)
    if i == 3: 
        neighbourhood.append(tag.text)
    i = i+1
    if i==4:
        i=1

In [45]:
canada=pd.DataFrame(columns=['Postalcode','Borough','Neighbourhood'])
canada['Postalcode'] = postcode
canada['Borough'] = borough
canada['Neighbourhood'] = neighbourhood
canada


,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n
7,M6A,North York,Lawrence Manor\n
8,M7A,Queen's Park,Not assigned\n
9,M8A,Not assigned,Not assigned\n


## Removing cells with no borough value

In [46]:
val_index=canada[canada['Borough']=='Not assigned'].index

In [47]:
canada.drop(canada.index[val_index],inplace=True)

In [48]:
canada.shape

(211, 3)

## Removing \n from Neighbourhood

In [49]:
canada['Neighbourhood'] = canada.Neighbourhood.str.replace("\n","")
canada = canada.reset_index(drop=True)
canada.head()

### If Neighbourhood is Not Assigned assign borough to Neighbourhood
neigh_index=canada[canada['Neighbourhood']=='Not assigned'].index

for i in range(211):
    if canada.Neighbourhood[i]=='Not assigned':
        canada.Neighbourhood[i] = canada.Borough[i]

In [50]:
canada

,Postalcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


## concating neighbourhoods with same postal code

In [51]:
canada = canada.groupby(['Postalcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
canada.head(10)

,Postalcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


## Getting Longitude,Latitude Coordinates

In [52]:
coordinates= pd.read_csv('https://cocl.us/Geospatial_data')

In [53]:
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [29]:
canada.shape

(103, 3)

In [55]:
## Merge the coordinates info in Postalcode/Borough/Neighborhood dataframe.
df = pd.merge(canada, coordinates,left_on='Postalcode',right_on='Postal Code',how='left')
df.drop('Postal Code', axis=1)



,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [57]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [58]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [59]:
#Foursquare Api credentials
CLIENT_ID = 'Q5RKWLCHMWVBQKCPK4XEGUFMAZBIQEUURXKOGB0MM1EHLYWF' # your Foursquare ID
CLIENT_SECRET = 'BJBJLWDDNJS44PDPEXVBDTRHDRVM1VWOVCF535Y5Q2QXQ3JP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Q5RKWLCHMWVBQKCPK4XEGUFMAZBIQEUURXKOGB0MM1EHLYWF
CLIENT_SECRET:BJBJLWDDNJS44PDPEXVBDTRHDRVM1VWOVCF535Y5Q2QXQ3JP


In [60]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=Q5RKWLCHMWVBQKCPK4XEGUFMAZBIQEUURXKOGB0MM1EHLYWF&client_secret=BJBJLWDDNJS44PDPEXVBDTRHDRVM1VWOVCF535Y5Q2QXQ3JP&ll=43.653963,-79.387207&v=20180605&radius=500&limit=100'

In [61]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d2f59a3c0c0840025c6e8c5'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 76,
  'suggestedBounds': {'ne': {'lat': 43.6584630045, 'lng': -79.38099903084075},
   'sw': {'lat': 43.649462995499995, 'lng': -79.39341496915925}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5227bb01498e17bf485e6202',
       'name': 'Downtown Toronto',
       'location': {'lat': 43.65323167517444,
        'lng': -79.38529600606677,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65323167517444,
          

In [62]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [63]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Japango,Sushi Restaurant,43.655268,-79.385165
2,Cafe Plenty,Café,43.654571,-79.389450
3,Textile Museum of Canada,Art Museum,43.654396,-79.386500
4,Poke Guys,Poke Place,43.654895,-79.385052


In [64]:
## create a function to repeat the same process to all the neighborhoods in Toronto

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [65]:
neigh_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [66]:
print(neigh_venues.shape)
neigh_venues.head()

(2249, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


In [67]:
## check how many venues were returned for each neighborhood
neigh_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",2,2,2,2,2,2
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",11,11,11,11,11,11
"Alderwood,Long Branch",9,9,9,9,9,9
"Bathurst Manor,Downsview North,Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",22,22,22,22,22,22
Berczy Park,57,57,57,57,57,57


## Analyze Each Neighborhood

In [68]:
# one hot encoding
canada_onehot = pd.get_dummies(neigh_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
canada_onehot['Neighborhood'] = neigh_venues['Neighborhood'] 

# move neighborhood column to the first column
column_list = canada_onehot.columns.tolist()
column_number = int(column_list.index('Neighborhood'))
column_list = [column_list[column_number]] + column_list[:column_number] + column_list[column_number+1:]
canada_onehot = canada_onehot[column_list]

canada_onehot.head()



,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0

In [69]:
canada_onehot.shape

(2249, 276)

In [70]:
## group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

canada_grouped = canada_onehot.groupby('Neighborhood').mean().reset_index()
canada_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.000,0.000000,0

In [71]:
canada_grouped.shape

(100, 276)

In [72]:
num_top_venues = 5

for hood in canada_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = canada_grouped[canada_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2              Bar  0.04
3  Thai Restaurant  0.04
4       Steakhouse  0.04


----Agincourt----
               venue  freq
0     Clothing Store  0.25
1     Breakfast Spot  0.25
2             Lounge  0.25
3       Skating Rink  0.25
4  Accessories Store  0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
               venue  freq
0         Playground   0.5
1               Park   0.5
2  Accessories Store   0.0
3  Mobile Phone Shop   0.0
4      Moving Target   0.0


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                 venue  freq
0        Grocery Store  0.18
1          Coffee Shop  0.09
2  Fried Chicken Joint  0.09
3  Japanese Restaurant  0.09
4           Beer Store  0.09


----Alderwood,Long Branch----
          venue  freq
0   Pizza Place  0.22
1   Coffee Shop  0.11
2           Gym  0.11
3  S

               venue  freq
0               Park  0.33
1  Convenience Store  0.33
2        Pizza Place  0.33
3  Mobile Phone Shop  0.00
4      Moving Target  0.00


----Emery,Humberlea----
                        venue  freq
0              Baseball Field   1.0
1           Accessories Store   0.0
2  Modern European Restaurant   0.0
3               Moving Target   0.0
4               Movie Theater   0.0


----Fairview,Henry Farm,Oriole----
                  venue  freq
0        Clothing Store  0.11
1           Coffee Shop  0.08
2  Fast Food Restaurant  0.08
3      Asian Restaurant  0.03
4           Bus Station  0.03


----First Canadian Place,Underground city----
              venue  freq
0       Coffee Shop  0.09
1              Café  0.07
2             Hotel  0.04
3        Steakhouse  0.04
4  Asian Restaurant  0.03


----Flemingdon Park,Don Mills South----
                   venue  freq
0             Beer Store  0.10
1            Coffee Shop  0.10
2       Asian Restaurant  0.10
3        

                  venue  freq
0           Coffee Shop  0.09
1        Clothing Store  0.06
2        Cosmetics Shop  0.04
3  Fast Food Restaurant  0.03
4                  Café  0.03


----Scarborough Village----
               venue  freq
0         Playground   1.0
1  Accessories Store   0.0
2  Mobile Phone Shop   0.0
3      Moving Target   0.0
4      Movie Theater   0.0


----Silver Hills,York Mills----
                             venue  freq
0                        Cafeteria   1.0
1                Accessories Store   0.0
2  Molecular Gastronomy Restaurant   0.0
3                           Museum   0.0
4                    Moving Target   0.0


----St. James Town----
                venue  freq
0         Coffee Shop  0.07
1               Hotel  0.05
2          Restaurant  0.05
3                Café  0.05
4  Italian Restaurant  0.04


----Stn A PO Boxes 25 The Esplanade----
                venue  freq
0         Coffee Shop  0.10
1                Café  0.04
2          Restaurant  0.04
3

In [73]:
## put that into a pandas dataframe
## First, write a function to sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## create the new dataframe and display the top 10 venues for each neighborhood.

In [74]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = canada_grouped['Neighborhood']

for ind in np.arange(canada_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(canada_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,Bar,Burger Joint,Asian Restaurant,Restaurant,American Restaurant,Bakery
1,Agincourt,Skating Rink,Breakfast Spot,Lounge,Clothing Store,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Playground,Park,Drugstore,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Fast Food Restaurant,Japanese Restaurant,Pharmacy,Coffee Shop,Sandwich Place,Discount Store,Beer Store,Fried Chicken Joint,Pizza Place
4,"Alderwood,Long Branch",Pizza Place,Coffee Shop,Skating Rink,Sandwich Place,Pub,Pool,Pharmacy,Gym,Comic Shop,Comfort Food Restaurant
5,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Deli / Bodega,Bank,Restaurant,Supermarket,Fried Chicken Joint,Frozen Yogurt Shop,Diner,Middle Eastern Restaurant,Sandwich Place
6,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
7,"Bedford Park,Lawrence Manor East",Italian Restaurant,Thai Restaurant,Coffee Shop,Fast Food Restaurant,Sushi Restaurant,Restaurant,Café,Indian Restaurant,Juice Bar,Pub
8,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Café,Cheese Shop,Seafood Restaurant,Bakery,Beer Bar,Steakhouse,Belgian Restaurant
9,"Birch Cliff,Cliffside West",College Stadium,Skating Rink,General Entertainment,Café,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Ethiopian Restaurant,Dumpling Restaurant,Dim Sum Restaurant


## Cluster Neighborhoods
## Run k-means to cluster the neighborhood into 5 clusters.

In [75]:
#set number of clusters
kclusters = 5

canada_grouped_clustering = canada_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(canada_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 3, 4, 3, 3, 3, 3, 3, 3, 3])

In [76]:
# add clustering labels
canada_merged = df.head(100)
canada_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
canada_merged = canada_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

canada_merged.head()

C:\Users\Ace\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Postalcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353,3,Fast Food Restaurant,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Colombian Restaurant
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",M1C,43.784535,-79.160497,3,Bar,Yoga Studio,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Diner
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",M1E,43.763573,-79.188711,4,Pizza Place,Rental Car Location,Medical Center,Breakfast Spot,Electronics Store,Mexican Restaurant,Intersection,Costume Shop,Dive Bar,Ethiopian Restaurant
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917,3,Coffee Shop,Mexican Restaurant,Korean Restaurant,Yoga Studio,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476,3,Bakery,Hakka Restaurant,Caribbean Restaurant,Athletics & Sports,Thai Restaurant,Bank,Fried Chicken Joint,Yoga Studio,Donut Shop,Dog Run


In [77]:
# sort the results by Cluster Labels
print(canada_merged.shape)
canada_merged.sort_values(["Cluster Labels"], inplace=True)
canada_merged

(100, 17)


,Postalcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
56,M5E,Downtown Toronto,Berczy Park,M5E,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Farmers Market,Café,Cheese Shop,Seafood Restaurant,Bakery,Beer Bar,Steakhouse,Belgian Restaurant
42,M4L,East Toronto,"The Beaches West,India Bazaar",M4L,43.668999,-79.315572,0,Park,Pizza Place,Brewery,Burger Joint,Burrito Place,Sandwich Place,Pub,Coffee Shop,Pet Store,Movie Theater
99,M9P,Etobicoke,Westmount,M9P,43.696319,-79.532242,1,Pizza Place,Coffee Shop,Intersection,Chinese Restaurant,Middle Eastern Restaurant,Sandwich Place,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Dim Sum Restaurant
88,M8V,Etobicoke,"Humber Bay Shores,Mimico South,New Toronto",M8V,43.605647,-79.501321,1,Fried Chicken Joint,American Restaurant,Restaurant,Café,Sandwich Place,Seafood Restaurant,Mexican Restaurant,Liquor Store,Coffee Shop,Fast Food Restaurant
93,M9A,Etobicoke,Islington Avenue,M9A,43.667856,-79.532242,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,M4K,East Toronto,"The Danforth West,Riverdale",M4K,43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Yoga Studio,Spa,Bookstore,Brewery,Bubble Tea Shop
26,M3B,North York,Don Mills North,M3B,43.745906,-79.352188,2,Café,Caribbean Restaurant,Gym / Fitness Center,Baseball Field,Japanese Restaurant,Yoga Studio,Donut Shop,Dive Bar,Dog Run,Doner Restaurant
70,M5X,Downtown Toronto,"First Canadian Place,Underground city",M5X,43.648429,-79.382280,2,Coffee Shop,Café,Hotel,Steakhouse,American Restaurant,Restaurant,Deli / Bodega,Bar,Bakery,Asian Restaurant
53,M5A,Downtown Toronto,"Harbourfront,Regent Park",M5A,43.654260,-79.360636,2,Coffee Shop,Bakery,Park,Café,Gym / Fitness Center,Pub,Mexican Restaurant,Restaurant,Breakfast Spot,Theater
55,M5C,Downtown Toronto,St. James Town,M5C,43.651494,-79.375418,3,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Bakery,Cosmetics Shop,Cocktail Bar,Breakfast Spot,Beer Bar


In [78]:
# create final map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(canada_merged['Latitude'], canada_merged['Longitude'], canada_merged['Neighbourhood'], canada_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

## Cluster 1

In [79]:
canada_merged.loc[canada_merged['Cluster Labels'] == 3, canada_merged.columns[[1] + list(range(5, canada_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
55,Downtown Toronto,-79.375418,3,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Bakery,Cosmetics Shop,Cocktail Bar,Breakfast Spot,Beer Bar
51,Downtown Toronto,-79.367675,3,Coffee Shop,Café,Restaurant,Italian Restaurant,Bakery,Pub,Pizza Place,Yoga Studio,Pet Store,Breakfast Spot
68,Downtown Toronto,-79.394420,3,Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Plane,Coffee Shop,Sculpture Garden,Boat or Ferry,Boutique,Bar
67,Downtown Toronto,-79.400049,3,Café,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Chinese Restaurant,Mexican Restaurant,Bar,Bakery,Dumpling Restaurant,Coffee Shop,Cocktail Bar
66,Downtown Toronto,-79.400049,3,Café,Bar,Japanese Restaurant,Bookstore,Restaurant,Bakery,Yoga Studio,Chinese Restaurant,Beer Bar,Beer Store
65,Central Toronto,-79.405678,3,Coffee Shop,Café,Sandwich Place,Pizza Place,American Restaurant,Indian Restaurant,BBQ Joint,Liquor Store,Burger Joint,Cosmetics Shop
64,Central Toronto,-79.411307,3,Jewelry Store,Park,Sushi Restaurant,Trail,Yoga Studio,Donut Shop,Discount Store,Dive Bar,Dog Run,Doner Restaurant
62,North York,-79.419750,3,Italian Restaurant,Thai Restaurant,Coffee Shop,Fast Food Restaurant,Sushi Restaurant,Restaurant,Café,Indian Restaurant,Juice Bar,Pub
61,Downtown Toronto,-79.379817,3,Coffee Shop,Hotel,Café,American Restaurant,Restaurant,Seafood Restaurant,Gym,Bakery,Steakhouse,Gastropub
60,Downtown Toronto,-79.381576,3,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Deli / Bodega,Gastropub,Bar,Gym,American Restaurant


## Cluster 2

In [80]:
canada_merged.loc[canada_merged['Cluster Labels'] == 1, canada_merged.columns[[1] + list(range(5, canada_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
99,Etobicoke,-79.532242,1,Pizza Place,Coffee Shop,Intersection,Chinese Restaurant,Middle Eastern Restaurant,Sandwich Place,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Dim Sum Restaurant
88,Etobicoke,-79.501321,1,Fried Chicken Joint,American Restaurant,Restaurant,Café,Sandwich Place,Seafood Restaurant,Mexican Restaurant,Liquor Store,Coffee Shop,Fast Food Restaurant
93,Etobicoke,-79.532242,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,East Toronto,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Yoga Studio,Spa,Bookstore,Brewery,Bubble Tea Shop


## Cluster 3

In [81]:
canada_merged.loc[canada_merged['Cluster Labels'] == 2, canada_merged.columns[[1] + list(range(5, canada_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,North York,-79.352188,2,Café,Caribbean Restaurant,Gym / Fitness Center,Baseball Field,Japanese Restaurant,Yoga Studio,Donut Shop,Dive Bar,Dog Run,Doner Restaurant
70,Downtown Toronto,-79.382280,2,Coffee Shop,Café,Hotel,Steakhouse,American Restaurant,Restaurant,Deli / Bodega,Bar,Bakery,Asian Restaurant
53,Downtown Toronto,-79.360636,2,Coffee Shop,Bakery,Park,Café,Gym / Fitness Center,Pub,Mexican Restaurant,Restaurant,Breakfast Spot,Theater


## Cluster 4

In [82]:
canada_merged.loc[canada_merged['Cluster Labels'] == 0, canada_merged.columns[[1] + list(range(5, canada_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
56,Downtown Toronto,-79.373306,0,Coffee Shop,Cocktail Bar,Farmers Market,Café,Cheese Shop,Seafood Restaurant,Bakery,Beer Bar,Steakhouse,Belgian Restaurant
42,East Toronto,-79.315572,0,Park,Pizza Place,Brewery,Burger Joint,Burrito Place,Sandwich Place,Pub,Coffee Shop,Pet Store,Movie Theater


## Cluster 5

In [83]:
canada_merged.loc[canada_merged['Cluster Labels'] == 4, canada_merged.columns[[1] + list(range(5, canada_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,-79.188711,4,Pizza Place,Rental Car Location,Medical Center,Breakfast Spot,Electronics Store,Mexican Restaurant,Intersection,Costume Shop,Dive Bar,Ethiopian Restaurant
46,Central Toronto,-79.405678,4,Coffee Shop,Clothing Store,Yoga Studio,Bagel Shop,Grocery Store,Fast Food Restaurant,Mexican Restaurant,Diner,Dessert Shop,Park
72,North York,-79.445073,4,Pizza Place,Park,Japanese Restaurant,Sushi Restaurant,Pub,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dessert Shop
13,Scarborough,-79.304302,4,Pizza Place,Pharmacy,Italian Restaurant,Bank,Thai Restaurant,Chinese Restaurant,Noodle House,Fast Food Restaurant,Fried Chicken Joint,Dumpling Restaurant
16,Scarborough,-79.205636,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,Central Toronto,-79.416936,4,Garden,Music Venue,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
79,North York,-79.490074,4,Basketball Court,Construction & Landscaping,Bakery,Park,Yoga Studio,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
89,Etobicoke,-79.543484,4,Pizza Place,Coffee Shop,Skating Rink,Sandwich Place,Pub,Pool,Pharmacy,Gym,Comic Shop,Comfort Food Restaurant
74,York,-79.453512,4,Park,Fast Food Restaurant,Women's Store,Market,Pharmacy,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
